### Module importation

In [1]:
import pandas as pd
import glob
import os
import psycopg2
from psycopg2 import sql
from pathlib import Path
import chardet
import datetime as dt
import numpy as np

### Encoding verification

In [2]:
file = 'info_stations/CatalogoEstaçõesAutomáticas.csv'

def encoding_detection(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

encodig = encoding_detection(file)

print(f'Encoding: {encodig}')


Encoding: ascii


### Preprocessing

In [3]:
cols_to_use = list(range(19))
rows_to_skip = list(range(9))

file_name = file
    
data = pd.read_csv(file, encoding=encodig, sep=";", skiprows=0)

cols_renamed = "DC_NOME;SG_ESTADO;CD_SITUACAO;VL_LATITUDE;VL_LONGITUDE;VL_ALTITUDE;DT_INICIO_OPERACAO;CD_ESTACAO"      
cols_renamed = cols_renamed.lower().split(";")

data.columns = cols_renamed

print("Original columns types:")
print(data.dtypes)

# conversão das colunas numéricas para valores numéricos
# obs: usando pd.to_numeric pode ocasionar erros principalmente devido ao sinal de negativo
for col in ['vl_latitude', 'vl_longitude', 'vl_altitude']:
    data[col] = data[col].str.replace(',', '.').astype(float)

# Conversão para o formato de data
data['dt_inicio_operacao'] = pd.to_datetime(data['dt_inicio_operacao'], format='%d/%m/%Y')

# validação, indicando quais colunas restaram e seus tipos, pós preprocessadas
print("\nPost processed columns types:")
print(data.dtypes)

data.to_csv('info_stations/station_info_preprocessed.csv', index=False)

Original columns types:
dc_nome               object
sg_estado             object
cd_situacao           object
vl_latitude           object
vl_longitude          object
vl_altitude           object
dt_inicio_operacao    object
cd_estacao            object
dtype: object

Post processed columns types:
dc_nome                       object
sg_estado                     object
cd_situacao                   object
vl_latitude                  float64
vl_longitude                 float64
vl_altitude                  float64
dt_inicio_operacao    datetime64[ns]
cd_estacao                    object
dtype: object
